In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.datasets import make_classification

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from targen.data import target

### Get random data

In [3]:
X, dummy = make_classification( n_samples=1000,
    n_features=20)

Map it to the dataframe

In [4]:
data = pd.DataFrame(X, columns = [f'col_{ix}' for ix in range(X.shape[1])])

In [5]:
data

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19
0,-1.850339,-1.197340,-0.355523,1.116181,1.845710,-0.477279,1.487521,0.315630,0.133985,-1.023797,-1.047755,0.650470,-0.318144,1.666031,0.175629,-1.099117,-0.756235,-0.506405,0.427088,-0.513930
1,1.488588,-0.913817,-1.418795,-0.747513,0.355358,-0.139962,-0.687829,-0.149273,0.551018,-1.487871,0.565381,-0.684635,0.463179,-1.374709,1.619136,-0.674355,-1.599199,0.909847,1.402835,-0.347821
2,-0.588405,-0.071284,-0.369446,0.751150,-1.082550,-0.731904,-0.194864,1.372475,-1.539740,0.300565,2.208001,-1.853615,0.578585,0.156055,-0.892241,-1.392123,-0.040691,0.309030,0.285427,1.147056
3,-1.701368,0.570917,-1.248805,-0.170574,-0.750793,0.629564,-2.122336,0.731701,-1.188198,0.706921,1.544067,-0.638208,-0.665960,0.821595,-0.600191,-1.040545,-0.306324,-1.203122,1.000677,0.704866
4,0.184303,1.012137,-1.679044,1.489328,-0.932593,0.857415,0.996254,0.730320,-0.476816,-1.924601,-0.056678,0.340880,0.488011,-1.848826,-1.419831,1.471304,-1.982000,0.447148,1.679173,-1.702966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.091892,-0.934044,0.147880,0.163007,-0.342986,0.627779,0.656880,-0.460088,0.406351,0.130552,-0.382486,0.691892,-0.353274,1.675991,-0.021521,0.697908,0.153288,0.252674,-0.143369,0.401495
996,-0.527461,0.150344,2.356521,-0.490172,0.008494,-0.276413,-0.612870,0.604112,0.552222,0.388602,-0.564388,-0.244958,-2.421771,0.261117,-0.513333,0.486842,1.518778,1.545145,-2.088257,-0.714567
997,1.365803,-1.024724,-0.898050,1.260115,-0.137339,-0.162091,0.728386,0.385492,1.142555,0.351636,0.138634,-0.906503,1.225850,0.016768,-0.606482,2.070227,-0.305880,0.913492,0.737808,0.051384
998,0.176144,-0.329279,1.793467,-1.074545,0.072454,0.907962,-0.346622,-1.672273,0.391016,0.568188,1.140742,0.298060,0.143206,1.991956,1.320334,-0.173261,1.304673,1.713957,-1.620925,-1.133157


# Define all expressions in one go and get the targets

Quick way to get your y target.<br>

Define the expression dictionary as in the example below.

Allowed keys are

- linear: expects a value being a string with an expression
- non_linear: expects a value being a string with an expression
- interaction: expects a value being a string with an expression
- conditional: expects a dictionary, where:<br>
    - keys: are the conditions
    - values are tuples:
        - the first element is the expression or a value to assign to in case when the conditions is true
        - the second element is the expression or a value to assign to in case when the conditions is false

- uniform noise: expects a dictionary with kwargs to apply, options are
    - weight
    - min (for scaling)
    - max (for scaling)

    
- gaussian noise: expects a dictionary with kwargs to apply, options are
    - weight
    - mu_gaus
    - sigma_gaus



In [6]:
expressions = {
    'linear': '-0.5*col_0 + 2.*col_1 + 0.7*col_7 -0.4*col_14',
    'non_linear': '-0.7*col_1**1.5 + 0.2*sin(col_7)+ 0.9*log(col_19) -0.1*col_13**2',
    'interaction': '0.05*col_3*col_4 -0.1*(col_7/col_11)',
    'conditional': {
        'col_0>0':(
            '-0.5*col_0 + 2.*col_1 + 0.7*col_7 -0.4*col_14', #True
            0 #False
        ),
        'col_2>0.15':(
            '-0.5*col_0 + 2.*col_13 + 0.2*col_17 -0.7*col_11',#True
            '-0.5*col_1' #False
        )
    },
    'uniform_noise': {
        'weight':0.5
    },
    'gaussian_noise': {
        'weight':0.5,
        'mu_gaus': 0
    }
}

In [7]:
contr_and_target_df = target.get_target_and_contributions(data, expressions=expressions, imbalance = 0.2)

In [8]:
contr_and_target_df.head()

,score_linear,score_non_linear,score_interaction,score_conditional,score_uniform_noise,score_gaussian_noise,score_total,y
0,0.818894,-0.975218,-0.062286,0.000000,-0.032489,0.036209,-0.214890,0
1,0.499113,-0.908762,-0.105083,0.499113,-0.457964,-0.267424,-0.741006,0
2,1.231402,-0.575233,-0.273727,0.000000,-0.135429,0.454203,0.701217,0
3,1.433956,-0.779449,-0.133892,0.000000,-0.219048,0.287103,0.588670,0
4,1.461646,-1.757218,-0.093105,1.461646,-0.348126,-0.434450,0.290392,0


In [9]:
contr_and_target_df['y'].value_counts()

0    800
1    200
Name: y, dtype: int64

# Build your own contributions 

This part descrives the inner workings of the `get_target_and_contributions` function.

### Simple contributions

Define here the contributions to use.<br>

Note the function `score_from_expressions`, this one can use any type of input.<br>

For documentation purposes, it is split in linear, non linear and interaction component, but they can be defined in one expression as well.

In [10]:
linear_component = '-0.5*col_0 + 2.*col_1 + 0.7*col_7 -0.4*col_14'
non_linear_component = '-0.7*col_1**1.5 + 0.2*sin(col_7)+ 0.9*log(col_19) -0.1*col_13**2'
inter_component = '0.05*col_3*col_4 -0.1*(col_7/col_11)'

In [11]:
score_linear = target.score_from_expression(data, expr=linear_component)
score_non_linear = target.score_from_expression(data, expr=non_linear_component)
score_interaction = target.score_from_expression(data, expr=inter_component)

### Conditional interaction

In [12]:
cond_expr = {
    'col_0>0':(
        '-0.5*col_0 + 2.*col_1 + 0.7*col_7 -0.4*col_14', #True
        0 #False
    ),
    'col_2>0.15':(
        '-0.5*col_0 + 2.*col_13 + 0.2*col_17 -0.7*col_11',#True
        '-0.5*col_1' #False
    )
}

In [13]:
cond_scores = target.score_with_condition(data,cond_expr = cond_expr )

In [14]:
cond_scores

0      0.000000
1      0.499113
2      0.000000
3      0.000000
4      1.461646
         ...   
995    0.000000
996    0.000000
997    0.661167
998    0.642566
999    0.000000
Length: 1000, dtype: float64

### Noise contribution

In [15]:
score_random_noise = target.score_uniform_noise(data, weight=0)
score_gaussian_noise = target.score_gaussian_noise(data, weight=0, mu_gaus=0.5)

### Put it all together and get the target

In [16]:
score_final = score_linear + score_non_linear + score_interaction + cond_scores + score_random_noise + score_gaussian_noise

In [17]:
y = target.define_target(score_final, imbalance_fraction=0.5)

y.value_counts()

1    500
0    500
dtype: int64

# Check the performances

In [18]:
from sklearn.metrics import roc_auc_score

In [19]:
roc_auc_score(y, score_final)

1.0

In [20]:
print(f'Linear AUC     : {np.round(roc_auc_score(y, score_linear),4)}')
print(f'Non Linear AUC : {np.round(roc_auc_score(y, score_non_linear),4)}')
print(f'Interaction AUC: {np.round(roc_auc_score(y, score_interaction),4)}')
print(f'Conditional AUC: {np.round(roc_auc_score(y, cond_scores),4)}')
print(f'Max AUC        : {np.round(roc_auc_score(y, score_linear + score_non_linear + score_interaction + cond_scores ),4)}')

Linear AUC     : 0.7025
Non Linear AUC : 0.6378
Interaction AUC: 0.4382
Conditional AUC: 0.8257
Max AUC        : 1.0


# Simple try

Add only linear contribution, build a Logisict Regression and see the interpretation



In [21]:
y_lin = target.define_target(score_linear, imbalance_fraction=0.5)
y_lin.value_counts()

1    500
0    500
dtype: int64

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [23]:
linear_component

'-0.5*col_0 + 2.*col_1 + 0.7*col_7 -0.4*col_14'

In [24]:
X_lin = MinMaxScaler().fit_transform(data[['col_0', 'col_1', 'col_7','col_14']])

In [25]:
lr = LogisticRegression(penalty='none', fit_intercept=False).fit(X_lin,y_lin)

In [26]:
roc_auc_score(y_lin,lr.predict_proba(X_lin)[:,1])

0.9398759999999998

In [27]:
lr.coef_, lr.intercept_

(array([[ -9.9421794 ,  17.88254454,   1.36980712, -11.23104604]]),
 array([0.]))

In [28]:
from scipy.stats import logistic
from scipy.special import expit

In [29]:
logistic.cdf(score_linear)[y_lin==1].min()

0.7332163442004278

In [30]:
logistic.cdf(score_linear)[y_lin==0].max()

0.7330079939503215